In [ ]:
!pip install langchainhub
!pip install langchain-qdrant
!pip install langchain transformers accelerate sentence-transformers

In [1]:
!git clone https://github.com/aalto-ui/chi21adaptive.git

Cloning into 'chi21adaptive'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 70 (delta 17), reused 53 (delta 10), pack-reused 0
Receiving objects: 100% (70/70), 370.94 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [16]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import JoplinLoader
from langchain_community.vectorstores.utils import filter_complex_metadata

loader = JoplinLoader(access_token="0437143735021d70121809d13b5bae5b3f3736ff125320e142b4945c03aee1e5cb57b28665bb53372054cea0beade9df2d39cca9cefd02c388eed46f4894f39e")


data = loader.load()
data = filter_complex_metadata(data)

In [17]:
for doc in data:
 old_path_with_txt_extension = doc.metadata["source"]
 new_path_without_txt_extension = old_path_with_txt_extension.replace(".txt", "")
 doc.metadata.update({"source": new_path_without_txt_extension})

In [18]:
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_community.vectorstores import Qdrant
from langchain_qdrant import Qdrant

In [19]:
model_name = "mxbai-embed-large"
embeddings = OllamaEmbeddings(model=model_name,
 show_progress=True,
 )

In [20]:
#Index new docs to db

qdrant = Qdrant.from_documents(
 data,
 embeddings,
 path="local_qdrant_joplin",
 collection_name="my_documents",
)

OllamaEmbeddings: 100%|███████████████████████████████████████████████████| 26/26 [00:22<00:00,  1.15it/s]


In [9]:
#Restore db

qdrant = Qdrant.from_existing_collection(
 embeddings,
 path="local_qdrant_joplin",
 collection_name="my_documents",
)

In [32]:
def pretty_print_docs(documents):
    for doc in documents:
        print(doc.metadata)
        print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")
        print(doc.page_content)


In [33]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.prompts.chat import HumanMessagePromptTemplate

# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)


In [34]:
query = "what are the groceries list in here"
found_docs = qdrant.similarity_search(query)
#pretty_print_docs(found_docs)

OllamaEmbeddings: 100%|█████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


In [35]:
found_docs

[Document(page_content='- [x] Yogert\n- [x] Milk\n- [ ]  Heavy cream\n- [x] Onions\n- [x] Bio Paper\n- [x] Toilet paper\n- [x] eggs\n- [x] Tomatoes\n- [x] Batteries\n- [x] pickled carrots\n- [x]  bread\n- [x]  chicken stock\n- [x]  shampoo\n- [x]  coffee\n- [x]  noodles\n- [ ]  nestle milk additive\n- [x]  shampoo\n- [ ]  Butter \n- [x]  Pickled Cabbbage\n- [ ]  vegetables\n- [x]  onion\n- [x]  broccoli\n- [x]  chicken\n- [x]  toothpaste\n- [ ]  cereal\n- [x]  pesto\n- [ ]  tortilla \n- [x]  earbuds\n- [x]  pasta\n- [x]  shower gell\n- [x]  ginger\n- [x]  oil\n- [x]  spinach\n- [ ]  cherry tomatoes\n- [ ]  garlic\n- [x]  sponge\n- [ ]  ketchup\n- [ ]  kitchen towel', metadata={'source': 'joplin://x-callback-url/openNote?id=a9ab7ef1f3b44decb242625f90d3708c', 'folder': 'To-Do', 'title': 'Groceries', 'created_time': '2023-07-01 02:28:20', 'updated_time': '2024-06-16 17:48:16', '_id': '98361c1702674799b9e88ebb8f51b639', '_collection_name': 'my_documents'}),
 Document(page_content='- [x] ch

### Implementation 1 (directly from serach docs and not from retreiver)

In [36]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
Prompt: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])
chain = prompt | llm | StrOutputParser()
#response = chain.invoke({"question":query,"context":found_docs})
#print(response)

In [37]:
for chunk in chain.stream({"question":query,"context":found_docs}):
    print(chunk, end='')

Based on the provided context, here is your groceries list:

* Yogurt
* Milk
* Onions
* Bio Paper
* Toilet paper
* Eggs
* Tomatoes
* Pickled carrots
* Bread
* Chicken stock
* Shampoo
* Coffee
* Noodles
* Butter
* Pickled cabbage
* Onion
* Broccoli
* Chicken
* Toothpaste
* Pesto
* Pasta
* Shower gel
* Ginger
* Oil
* Spinach

### Implementation 2 (using retreiver)

In [9]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [10]:
from langchain import hub



retriever = MultiQueryRetriever.from_llm(
    qdrant.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = hub.pull("rlm/rag-prompt")
Prompt: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])


In [11]:
# Define templates for prompts
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from typing import List, Tuple
from langchain.schema import format_document

#Initialte chat_history

chat_history = []


# Create a memory instance
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question", memory_key="chat_history"
)

# Define steps for the chain
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"),
)



# Define templates for prompts
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""


ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "HumanMessage: " + dialogue_turn[0]
        ai = "AIMessage: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer




def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)




standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: x["chat_history"],        
#        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}

retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}

# Create the final chain by combining the steps
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [12]:
#stream chain 7

input = """
Is there any diffrence between policy network and value network? why are these used in here?
"""
inputs = {"question": input, "chat_history": chat_history}



chunks = []
chunks_answer = []
for chunk in final_chain.stream(inputs):
    chunks.append(chunk)
    if 'answer' in chunk:
        print(chunk['answer'].content, end='')
        chunks_answer.append(chunk['answer'].content)
    else:
        pass


#Below code might not work


# Save the conversation in memory
#generated_answer = chunks['answer']

from langchain.schema.messages import HumanMessage, AIMessage

chat_history.extend([
    HumanMessage(content=input),
    AIMessage(content=chunks_answer),
    #AIMessage(content=result["answer"].content),
])


# Load memory to see the conversation history
memory.load_memory_variables({})

#memory.save_context(inputs, {"answer": generated_answer.content})
memory.save_context(inputs, {"answer": chunks_answer})

OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.62it/s]


A policy network and a value network are both neural network models used in reinforcement learning. However, they serve different purposes and have distinct architectures.

Policy Networks:

Policy networks, also known as actor-critic networks or policy-value networks, are designed to learn the optimal policy for a given task. The goal of a policy network is to maximize the cumulative reward over an episode, while exploring different actions and their corresponding outcomes. In other words, the policy network learns to select the best action given the current state of the environment.

The architecture of a policy network typically consists of two main components: an actor and a critic. The actor generates actions based on the current state of the environment, while the critic evaluates the quality of the selected action. The output of the actor is used to make decisions in the real-world environment, while the output of the critic provides a value estimate for each possible action.

V